In [139]:
import pandas as pd
import numpy as np

## Reading in the Datasets

#### - Concatenating all course data into one large dataset
#### - Merging by the ppsk (unique student ID) to get demographics info

In [140]:
courses_2012 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2012.txt", sep="\t", low_memory = False)
courses_2013 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2013.txt", sep="\t", low_memory = False)
courses_2014 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2014.txt", sep="\t", low_memory=False)
courses_2015 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2015.txt", sep="\t", low_memory = False)
courses_2016 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2016.txt", sep="\t", low_memory = False)
courses_2017 = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.student.course.data.2017.txt", sep="\t", low_memory = False)
ppsk = pd.read_csv("/Users/shreya/Desktop/Research/uc.berkeley.stem.pathways.2007.2018.txt", sep="\t", low_memory=False)

In [141]:
#concatenating the course data
course_data = pd.concat([courses_2012, courses_2013, courses_2014, courses_2015, courses_2016, courses_2017])


In [142]:
#merging by ppsk
student_data = pd.merge(course_data, ppsk, on = 'ppsk', how='left')


## Filtering the DataFrame

### - Filter by lower division CS --> (61A, 61B, 70)
### - Make dummy variables
### - Group by ppsk
### - Get average gpa of lower divs

In [152]:
#lower div
student_data = student_data[student_data['undergrad.status']=='Undergraduate']
cs_lower = student_data[student_data['course.number'].isin(['61A', '61B', '70'])]

#turn final.majors into separate tables and turn into one column
fmm1 = pd.DataFrame(data = {'major': cs_lower['final.major.major.1']})
fmm2 = pd.DataFrame(data = {'major': cs_lower['final.major.major.2']})
fmm3 = pd.DataFrame(data = {'major': cs_lower['final.major.major.3']})
fmm4 = pd.DataFrame(data = {'major': cs_lower['final.major.major.4']})

fmm_1 = pd.DataFrame(data = {'major': cs_lower['first.major.major.1']})
fmm_2 = pd.DataFrame(data = {'major': cs_lower['first.major.major.2']})
fmm_3 = pd.DataFrame(data = {'major': cs_lower['first.major.major.3']})
fmm_4 = pd.DataFrame(data = {'major': cs_lower['first.major.major.4']})

declared_majors = fmm_1.combine_first(fmm_2).combine_first(fmm_3).combine_first(fmm_4).combine_first(fmm2).combine_first(fmm3).combine_first(fmm4)


#add column to cs_lower
cs_lower = pd.merge(cs_lower, declared_majors, left_on = 'ppsk', right_index = True, how='left')


#create dummies
major_dummies = pd.get_dummies(cs_lower[['major']])
# modified_cs = pd.get_dummies(cs_lower[['ppsk', 'instructor.name', 'year.name', 'course.number', 'gender', 'ethnic.l1']]).drop_duplicates() #flexible
modified_cs = pd.get_dummies(cs_lower[['ppsk','course.number', 'gender', 'ethnic.l1']]).drop_duplicates() #flexible


modified_cs



,ppsk,course.number_61A,course.number_61B,course.number_70,gender_Decline to State,gender_Female,gender_Male,ethnic.l1_Asian/Pacific Islander,ethnic.l1_International,ethnic.l1_Underrepresented Minority,ethnic.l1_White/Other
569,847130,0,1,0,0,0,1,1,0,0,0
1572,916674,0,1,0,0,0,1,0,0,0,1
1649,920733,0,1,0,0,0,1,1,0,0,0
1952,937794,0,1,0,0,0,1,0,0,0,1
2247,948499,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2881000,3124335,1,0,0,0,0,0,0,0,0,0
2881017,3125025,0,0,1,0,0,0,0,0,0,0
2881048,3127538,1,0,0,0,0,0,0,0,0,0
2881150,3131520,0,1,0,0,0,0,0,0,0,0


In [153]:
#group by ppsk

grouped_courses = modified_cs.groupby('ppsk').agg(max)
# add year, major declaration

new_df = pd.DataFrame(data = { 'ppsk': cs_lower['ppsk'], 'declared': major_dummies['major_L&S Computer Science']}).drop_duplicates()
grouped_courses = grouped_courses.merge(new_df, on = 'ppsk', how = 'left')


#add gpa to modified_cs
gpa = pd.DataFrame(data = cs_lower.groupby(['ppsk'])['grade.points'].mean())

final_table = grouped_courses.merge(gpa, on='ppsk', how = 'left')

In [158]:
final_table[final_table['declared'] == 1]
# sum(final_table['declared'])

,ppsk,course.number_61A,course.number_61B,course.number_70,gender_Decline to State,gender_Female,gender_Male,ethnic.l1_Asian/Pacific Islander,ethnic.l1_International,ethnic.l1_Underrepresented Minority,ethnic.l1_White/Other,declared,grade.points
225,936738,1,0,0,0,0,1,0,0,0,1,1,NaN
237,939307,0,0,1,0,0,1,1,0,0,0,1,2.000000
468,974520,0,1,0,0,0,1,1,0,0,0,1,3.300000
544,982024,0,1,1,0,0,1,1,0,0,0,1,1.466667
2973,1308469,1,0,0,0,1,0,0,1,0,0,1,NaN
4061,1316181,1,1,1,0,0,0,0,0,0,0,1,3.000000
4453,1473221,1,0,1,0,0,1,1,0,0,0,1,3.850000
4596,1473730,1,1,1,0,0,1,0,1,0,0,1,2.666667
4610,1473775,1,0,0,0,1,0,0,0,0,1,1,NaN
4737,1474444,0,1,1,0,0,1,1,0,0,0,1,3.000000


In [155]:
final_table

,ppsk,course.number_61A,course.number_61B,course.number_70,gender_Decline to State,gender_Female,gender_Male,ethnic.l1_Asian/Pacific Islander,ethnic.l1_International,ethnic.l1_Underrepresented Minority,ethnic.l1_White/Other,declared,grade.points
0,7780,1,0,0,0,0,0,0,0,0,0,0,1.15
1,136248,1,0,0,0,0,0,0,0,0,0,0,NaN
2,294778,0,0,1,0,0,0,0,0,0,0,0,2.70
3,360766,1,0,0,0,0,0,0,0,0,0,0,0.00
4,434810,0,0,1,0,0,0,0,0,0,0,0,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15209,3134805,1,0,0,0,0,1,0,0,0,1,0,2.70
15210,3135896,1,0,0,0,0,0,0,0,0,0,0,2.30
15211,3136455,1,0,0,0,0,0,0,0,0,0,0,4.00
15212,3137103,1,0,0,0,0,0,0,0,0,0,0,4.00
